In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
from PIL import Image

from mapsgan import Trajectories
from mapsgan.modules import *
%load_ext autoreload
%autoreload 2

In [2]:
data = Trajectories('/mnt/Clouds/MapsGAN/data/eth/test')

In [3]:
socen = SocialEncoder()
#socatt = SocialAttention()
phyen = PhysicalEncoder('resnet')
att = Attention()
#phyatt = PhysicalAttention()
decoder = DecoderGAN(input_dim=544)
discriminator = Discriminator()

In [10]:
hidden = decoder.init_hidden()

sorted_seq = socen(data[0]['train'], data[0]['distances'])

In [7]:
img = Image.open('../../data/eth/eth_static.jpg')

transform = transforms.Compose([transforms.ToTensor()
                    ,transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])

resnet_feat = phyen(transform(img))

In [17]:
generated = torch.zeros(12, 32, 2)
for i in range(12):
    att_feats = att(sorted_seq, resnet_feat, hidden)
    hidden, out = decoder(att_feats)
    generated[i] = out

In [25]:
generated.shape, data[0]['groundtruth'].shape

(torch.Size([12, 32, 2]), torch.Size([12, 32, 2]))

In [72]:
randtruth = torch.Tensor(32).uniform_().bernoulli()

In [76]:
torch.cat((data[0]['train'], generated[:,agent,:]), dim = 0)

RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 3 and 2 at /opt/conda/conda-bld/pytorch-cpu_1532578932944/work/aten/src/TH/generic/THTensorMath.cpp:3607

In [80]:
data[0]['train'].shape, generated.shape

(torch.Size([8, 32, 2]), torch.Size([12, 32, 2]))

0

In [124]:
trajectories = torch.zeros(20, 32, 2)
for agent, val in enumerate(randtruth):
    if val == 0.:
        trajectories[:,agent] = torch.cat((data[0]['train'][:, agent], generated[:,agent]), dim = 0)
    else:
        trajectories[:,agent] = torch.cat((data[0]['train'][:, agent], data[0]['groundtruth'][:, agent]), dim = 0)
        

In [127]:
randtruth

tensor([0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
        1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0.])

In [129]:
trajectories[:, 2]

tensor([[0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.1242],
        [0.0000, 0.1221],
        [0.0000, 0.1222],
        [0.0000, 0.1192],
        [0.0000, 0.1213],
        [0.0000, 0.1223],
        [0.0000, 0.1255],
        [0.0000, 0.1206],
        [0.0000, 0.1228],
        [0.0000, 0.1272],
        [0.0000, 0.1239],
        [0.0000, 0.1242]], grad_fn=<SelectBackward>)

In [47]:
discriminator(out)

tensor([[0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608],
        [0.0608]], grad_fn=<ThAddmmBackward>)

In [44]:
data[0]['lossmask'].shape

torch.Size([20, 32])

In [ ]:
class Solver():
    default_adam_args = {"lr": 1e-3,
                     "betas": (0.9, 0.999),
                     "eps": 1e-8,
                     "weight_decay": 0.0}

    def __init__(self, optim=torch.optim.Adam, optim_args={},
                 loss_func=torch.nn.CrossEntropyLoss()):
        self.optim_args = self.default_adam_args
        if optim_args:
            self.optim_args = optim_args
        self.optim = optim
        self.loss_func = loss_func
        self._reset_histories()

    def _reset_histories(self):
        """
        Resets train and val histories for the accuracy and the loss.
        """
        self.train_loss_history = []
        self.train_acc_history = []
        self.test_loss_history = []
        self.test_acc_history = []
        self.best_acc = 0

    def train(self, models, train_loader, epoch):
        """
        Train a given model with the provided data.
        """
        model.train()
        optim = self.optim(filter(lambda p: p.requires_grad, model.parameters()), **self.optim_args)

        if torch.cuda.is_available():
            model.cuda()

        print('\nEpoch: %d' % epoch)
        train_loss = 0
        correct = 0
        total = 0
        for batch_idx, (targets, inputs) in enumerate(train_loader, 1):
            
            optim.zero_grad()
                            
            #if model.is_cuda:
            #    inputs = inputs.cuda()
            #    targets = targets.cuda()
                
            outputs = model(inputs)['layer8'].squeeze()
            
            loss = self.loss_func(outputs, targets)
                            
            loss.backward()
            optim.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            self.train_loss_history.append(loss.data.cpu().numpy())
            self.train_acc_history.append(100.*correct/total)
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))